Ide: tag dansk-sproget wikipedia, find alle brugere og lav netværk over brugere, der har redigeret de samme artikler. Derefter, få lix-tal eller lignende fra alle artikler, og se om der er sammenhæng... Eller groupér efter konnektivitet, og se på sammenhængen i grupperne baseret på keywords i artiklerne, eller fordelingen af kategorier.

X Filter navne med bot eller ip-adresser.
X Få data
X Konstruer graf
X Cluster
O Test clusters for modularity mod random graphs
O Test text similarity mellem og indenfor clusters
O Find kategorier for clusters

In [75]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import chain
from nltk import word_tokenize
from itertools import combinations
from functools import reduce
from collections import defaultdict
import re
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn import metrics
import pywikibot
import networkx as nx
from tqdm import tqdm
from joblib import Parallel, delayed
from pywikibot import pagegenerators
import nltk
from datetime import datetime, timedelta
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from netwulf import visualize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import community as community_louvain

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Getting data

In [2]:
def isbot(string):
    return 'bot' in string.lower()

def get_user_contributions(username):
    site = pywikibot.Site('da', 'wikipedia')
    user = pywikibot.User(site, username)
    contributions = user.contributions(total=500)
    articles = [contrib[0].title() for contrib in contributions 
                if ':' not in contrib[0].title()]
    return articles

def get_page_contributors(pagename):
    site = pywikibot.Site('da', 'wikipedia')
    page = pywikibot.Page(site, pagename)
    contributors = page.contributors()
    # Implement botfilter
    usernames = [user for user in contributors if isbot(user)]
    return {pagename: usernames}

In [3]:
site = pywikibot.Site('da', 'wikipedia')
all_pages_gen = site.allpages()
def get_page_info(page):
    contributors = list(page.contributors())
    bots = [c for c in contributors if isbot(c)]
    return {'name': page.title(), 'bots': bots, 'categories': list(page.categories())}

parallel = Parallel(n_jobs=-2, return_as = "generator")
data = parallel(delayed(get_page_info)(page)
                               for i, page in enumerate(all_pages_gen))
page_data = []
counter = 0
print_every = 10
break_when = 1000
while True:
    try:
        page = next(data)
        if not page['bots'] == []:
            page_data += [page]
        counter += 1
        if counter % print_every == 0:
            print(counter)
        if counter == break_when:
            break
    except StopIteration:
        print('Done')
        break
        
pages_dataframe = pd.DataFrame(page_data)
pd.to_pickle(pages_dataframe, 'pages_dataframe')
pages_dataframe

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000


,name,bots,categories
0,!!!,"[Steenthbot, MGA73bot, Addbot, Luckas-bot, Luc...",[[[da:Kategori:Artikler med musikhenvisninger ...
1,"""Fjerne Israel fra landkortet""- kontroversen",[Xqbot],[]
2,"""Fjerne Israel fra landkortet""-bemærkningen","[Xqbot, Broadbot]",[]
3,"""Gråt"" rumvæsen",[Xqbot],[]
4,"""Heroes""","[InternetArchiveBot, MGA73bot, Steenthbot, Dip...","[[[da:Kategori:Album fra 1977]], [[da:Kategori..."
...,...,...,...
780,1. division i ishockey 1975-76,[InternetArchiveBot],"[[[da:Kategori:DM i ishockey]], [[da:Kategori:..."
781,1. division i ishockey 1976-77,[InternetArchiveBot],"[[[da:Kategori:DM i ishockey]], [[da:Kategori:..."
782,1. division i ishockey 1977-78,[InternetArchiveBot],"[[[da:Kategori:DM i ishockey]], [[da:Kategori:..."
783,1. division i ishockey 1978-79,[InternetArchiveBot],"[[[da:Kategori:DM i ishockey]], [[da:Kategori:..."


In [47]:
pages_dataframe['categories'] = pages_dataframe['categories'].apply(lambda categories: [category.title()[9:] for category in categories])

In [5]:
bots_dataframe = pd.DataFrame(pages_dataframe['bots'].explode().value_counts())
bots_dataframe = bots_dataframe.rename_axis('name')
bots_dataframe = bots_dataframe.reset_index()
pd.to_pickle(bots_dataframe, 'bots_dataframe')
bots_dataframe

,name,count
0,Steenthbot,634
1,Addbot,415
2,Xqbot,378
3,WeggeBot,365
4,Luckas-bot,339
...,...,...
155,Le Pied-bot~dawiki,1
156,AsgardBot,1
157,NjardarBot,1
158,WinstonBot,1


In [7]:
def get_bot_data(bot_name):
    site = pywikibot.Site('da', 'wikipedia')
    bot = pywikibot.User(site, bot_name)
    edit_count = bot.editCount()
    return {'edit_count': edit_count}

bots_dataframe = pd.concat([bots_dataframe, bots_dataframe['name'].apply(lambda x: pd.Series(get_bot_data(x)))], axis=1)
bots_dataframe

,name,count,edit_count
0,Steenthbot,634,751514
1,Addbot,415,151154
2,Xqbot,378,137057
3,WeggeBot,365,67893
4,Luckas-bot,339,194041
...,...,...,...
155,Le Pied-bot~dawiki,1,1922
156,AsgardBot,1,36
157,NjardarBot,1,2912
158,WinstonBot,1,28


In [85]:
p = next(iter(all_pages_gen))
print(p.title())
list(p.categories())

470'erne f.Kr.


[Category('Kategori:5. århundrede f.Kr.'), Category('Kategori:Årtier')]

In [40]:
list(pages_dataframe['categories'][0])

[Category('Kategori:Artikler med musikhenvisninger fra Wikidata'),
 Category('Kategori:Commons-kategori på Wikidata er ens med sidetitel'),
 Category('Kategori:Lokalt billede forskelligt fra Wikidata'),
 Category('Kategori:Musikgrupper fra USA'),
 Category('Kategori:Påbegyndte artikler (samlet liste)'),
 Category('Kategori:Påbegyndte artikler om musikgrupper fra USA'),
 Category('Kategori:Wikipedia artikler med BNF autoritetsdata-ID'),
 Category('Kategori:Wikipedia artikler med GND autoritetsdata-ID'),
 Category('Kategori:Wikipedia artikler med ISNI autoritetsdata-ID'),
 Category('Kategori:Wikipedia artikler med LCCN autoritetsdata-ID'),
 Category('Kategori:Wikipedia artikler med MusicBrainz autoritetsdata-ID'),
 Category('Kategori:Wikipedia artikler med VIAF autoritetsdata-ID')]

### Dataframes

### Graph construction

In [15]:
edge_dict = defaultdict(lambda: {'pages':[],'n_pages':0})

def page_edges(page):
    contributors = page['bots']
    if len(contributors) < 2:
        return []
    contributors.sort()
    contributor_pairs = list(combinations(contributors, r=2))
    for contributor_pair in contributor_pairs:
        edge_dict[contributor_pair]['pages'].append(page['name'])
        edge_dict[contributor_pair]['n_pages'] += 1

pages_dataframe.apply(page_edges, axis=1)
G = nx.Graph()
G.add_edges_from(edge_dict.keys())
nx.set_edge_attributes(G, edge_dict)

In [17]:
len(bots_dataframe)

160

In [18]:
len(G.edges)

6824

In [20]:
nx.get_edge_attributes(G, 'n_pages')

{('Addbot', 'LucienBOT'): 86,
 ('Addbot', 'Luckas-bot'): 325,
 ('Addbot', 'MGA73bot'): 80,
 ('Addbot', 'Steenthbot'): 385,
 ('Addbot', 'Broadbot'): 85,
 ('Addbot', 'VolkovBot'): 293,
 ('Addbot', 'Alecs.bot'): 9,
 ('Addbot', 'LaaknorBot'): 32,
 ('Addbot', 'RedBot'): 36,
 ('Addbot', 'SieBot'): 300,
 ('Addbot', 'TXiKiBoT'): 317,
 ('Addbot', 'Thijs!bot'): 332,
 ('Addbot', 'WeggeBot'): 347,
 ('Addbot', 'Zorrobot'): 35,
 ('Addbot', 'AlleborgoBot'): 73,
 ('Addbot', 'Bot-Schafter'): 5,
 ('Addbot', 'BotMultichill'): 59,
 ('Addbot', 'ChuispastonBot'): 196,
 ('Addbot', 'DumZiBoT'): 8,
 ('Addbot', 'EmausBot'): 288,
 ('Addbot', 'FlaBot'): 178,
 ('Addbot', 'FribergBot'): 2,
 ('Addbot', 'InternetArchiveBot'): 74,
 ('Addbot', 'JAnDbot'): 302,
 ('Addbot', 'LinkFA-Bot'): 6,
 ('Addbot', 'Louperibot'): 81,
 ('Addbot', 'MastiBot'): 16,
 ('Addbot', 'MerlIwBot'): 71,
 ('Addbot', 'Nallimbot'): 14,
 ('Addbot', 'Nobot~dawiki'): 212,
 ('Addbot', 'RibotBOT'): 12,
 ('Addbot', 'RobotQuistnix'): 18,
 ('Addbot', 'Sil

### Graph analysis

In [83]:
np.mean(list(dict(G.degree).values()))
np.log(len(G.nodes))

5.056245805348308

In [84]:
pages_dataframe

,name,bots,categories,communities,tokens
0,!!!,"[Addbot, LucienBOT, Luckas-bot, MGA73bot, Stee...","[Artikler med musikhenvisninger fra Wikidata, ...","[0, 2]","[artikler, musikhenvisninger, wikidata, common..."
1,"""Fjerne Israel fra landkortet""- kontroversen",[Xqbot],[],[0],[]
2,"""Fjerne Israel fra landkortet""-bemærkningen","[Broadbot, Xqbot]",[],[0],[]
3,"""Gråt"" rumvæsen",[Xqbot],[],[0],[]
4,"""Heroes""","[Dipsacus fullonum bot, InternetArchiveBot, MG...","[Album fra 1977, David Bowie-album, Påbegyndte...",[0],"[album, david, bowie-album, påbegyndte, artikl..."
...,...,...,...,...,...
780,1. division i ishockey 1975-76,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1975, Ishockey i 19...",[0],"[dm, ishockey, ishockey, ishockey, sider, weba..."
781,1. division i ishockey 1976-77,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1976, Ishockey i 19...",[0],"[dm, ishockey, ishockey, ishockey, sider, weba..."
782,1. division i ishockey 1977-78,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1977, Ishockey i 19...",[0],"[dm, ishockey, ishockey, ishockey, sider, weba..."
783,1. division i ishockey 1978-79,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1978, Ishockey i 19...",[0],"[dm, ishockey, ishockey, ishockey, sider, weba..."


### Clustering

In [22]:
partition = community_louvain.best_partition(G)

In [38]:
n_communities = max(partition.values()) + 1

In [39]:
def communities_of_page(bots):
    communities = []
    for bot in bots:
        try:
            communities.append(partition[bot])
        except KeyError:
            pass
    return list(set(communities))

pages_dataframe['communities'] = pages_dataframe['bots'].apply(communities_of_page)
pages_dataframe

,name,bots,categories,communities
0,!!!,"[Addbot, LucienBOT, Luckas-bot, MGA73bot, Stee...",[[[da:Kategori:Artikler med musikhenvisninger ...,"[0, 2]"
1,"""Fjerne Israel fra landkortet""- kontroversen",[Xqbot],[],[0]
2,"""Fjerne Israel fra landkortet""-bemærkningen","[Broadbot, Xqbot]",[],[0]
3,"""Gråt"" rumvæsen",[Xqbot],[],[0]
4,"""Heroes""","[Dipsacus fullonum bot, InternetArchiveBot, MG...","[[[da:Kategori:Album fra 1977]], [[da:Kategori...",[0]
...,...,...,...,...
780,1. division i ishockey 1975-76,[InternetArchiveBot],"[[[da:Kategori:DM i ishockey]], [[da:Kategori:...",[0]
781,1. division i ishockey 1976-77,[InternetArchiveBot],"[[[da:Kategori:DM i ishockey]], [[da:Kategori:...",[0]
782,1. division i ishockey 1977-78,[InternetArchiveBot],"[[[da:Kategori:DM i ishockey]], [[da:Kategori:...",[0]
783,1. division i ishockey 1978-79,[InternetArchiveBot],"[[[da:Kategori:DM i ishockey]], [[da:Kategori:...",[0]


In [48]:
pages_dataframe

,name,bots,categories,communities
0,!!!,"[Addbot, LucienBOT, Luckas-bot, MGA73bot, Stee...","[Artikler med musikhenvisninger fra Wikidata, ...","[0, 2]"
1,"""Fjerne Israel fra landkortet""- kontroversen",[Xqbot],[],[0]
2,"""Fjerne Israel fra landkortet""-bemærkningen","[Broadbot, Xqbot]",[],[0]
3,"""Gråt"" rumvæsen",[Xqbot],[],[0]
4,"""Heroes""","[Dipsacus fullonum bot, InternetArchiveBot, MG...","[Album fra 1977, David Bowie-album, Påbegyndte...",[0]
...,...,...,...,...
780,1. division i ishockey 1975-76,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1975, Ishockey i 19...",[0]
781,1. division i ishockey 1976-77,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1976, Ishockey i 19...",[0]
782,1. division i ishockey 1977-78,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1977, Ishockey i 19...",[0]
783,1. division i ishockey 1978-79,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1978, Ishockey i 19...",[0]


In [63]:
stop_words = set(stopwords.words('danish'))
def tokenize_category_list(category_list, exclude_stop=True):
    text = ' '.join(category_list)
    text = text.lower()
    text = ''.join([char for char in text if (char.isalpha() or char in ' -')])
    tokens = word_tokenize(text)
    if exclude_stop:
        return [token for token in tokens if not token in stop_words]
    return tokens

In [64]:
pages_dataframe['tokens'] = pages_dataframe['categories'].apply(tokenize_category_list)
pages_dataframe

,name,bots,categories,communities,tokens
0,!!!,"[Addbot, LucienBOT, Luckas-bot, MGA73bot, Stee...","[Artikler med musikhenvisninger fra Wikidata, ...","[0, 2]","[artikler, musikhenvisninger, wikidata, common..."
1,"""Fjerne Israel fra landkortet""- kontroversen",[Xqbot],[],[0],[]
2,"""Fjerne Israel fra landkortet""-bemærkningen","[Broadbot, Xqbot]",[],[0],[]
3,"""Gråt"" rumvæsen",[Xqbot],[],[0],[]
4,"""Heroes""","[Dipsacus fullonum bot, InternetArchiveBot, MG...","[Album fra 1977, David Bowie-album, Påbegyndte...",[0],"[album, david, bowie-album, påbegyndte, artikl..."
...,...,...,...,...,...
780,1. division i ishockey 1975-76,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1975, Ishockey i 19...",[0],"[dm, ishockey, ishockey, ishockey, sider, weba..."
781,1. division i ishockey 1976-77,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1976, Ishockey i 19...",[0],"[dm, ishockey, ishockey, ishockey, sider, weba..."
782,1. division i ishockey 1977-78,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1977, Ishockey i 19...",[0],"[dm, ishockey, ishockey, ishockey, sider, weba..."
783,1. division i ishockey 1978-79,[InternetArchiveBot],"[DM i ishockey, Ishockey i 1978, Ishockey i 19...",[0],"[dm, ishockey, ishockey, ishockey, sider, weba..."


In [73]:
corpus = pages_dataframe[['communities','tokens']].explode('communities').groupby('communities')['tokens'].apply(lambda x: list(chain(*x)))
corpus

communities
0    [artikler, musikhenvisninger, wikidata, common...
1    [intet, lokalt, billede, intet, billede, wikid...
2    [artikler, musikhenvisninger, wikidata, common...
3    [intet, lokalt, billede, intet, billede, wikid...
Name: tokens, dtype: object

In [74]:
# Returns top k terms as defined by TF-IDF
def top_k_terms_TF_IDF(tokens, k, IDF_dict):
    tokens, counts = np.unique(tokens, return_counts=True)
    tfidf = np.zeros(len(counts))
    for i, (token, count) in enumerate(list(zip(tokens, counts))):
        tfidf[i] = count * IDF_dict[token]
    top_k_idxs = np.argsort(tfidf)[:-(k+1):-1]
    return tokens[top_k_idxs]

# For the whole corpus define the IDF value for each term and return
# the results as a dictionary: {term:IDF}
def IDF(corpus):
    IDF_dict = defaultdict(lambda: 0)
    N = len(corpus)
    for doc in corpus:
        for term in list(set(doc)):
            IDF_dict[term] += 1
    return {term: np.log(N / count) for term, count in IDF_dict.items()}

IDF_dict = IDF(corpus)
TFIDF_df = pd.DataFrame(data={c: top_k_terms_TF_IDF(corpus[c], 10, IDF_dict) for c in range(4)})
TFIDF_df

,0,1,2,3
0,ishockey,administrationskategori,norge,historie
1,superisligaen,tekstkilder,basketball,spaniens
2,dm,kortfilm,basketball-ligaer,militær
3,administrationskategori,dokumentarfilm,kvinder,solsystemet
4,stumfilm,flertydig,skydevåben,borgerkrige
5,floorball,tyske,foreninger,hæren
6,sprog,breddegrader,metallica-sange,bruger
7,sovjetunionen,fly,brahe,bydele
8,dokumentarfilm,flertydige,sundhed,arrondissement
9,tekstkilder,længde-,norske,timeline
